<a href="https://colab.research.google.com/github/marisageyer/NASSP_2024/blob/master/2024_NASSP_MeerKATdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MeerKAT Pulsar data tutorial
### by  Marisa Geyer
    
-------








# &star; &star; &#128269; $\quad$Data detective work $\quad$&star; &star; &star;

It is Thursday afternoon and unanalysed MeerKAT data from a distant source has landed on your desk. 


You roll up your sleeves.

It is time to dig in and find what mysteries it is hiding.

## Step 1: Getting your toolkits ready
Luckily this is not your first day on job. As a seasoned astronomer, you already suspect which tools will be most useful to you. You compile a list of software to load in, and press GO!

**Action 1:** Load the below cells to install the necessary software tools to use on the newly received data

In [ ]:
# from https://pypi.org/project/condacolab/
!pip install -q condacolab
import condacolab
condacolab.install()

**Note:** The next step of installing psrchive can take a little while **(~3 to 5min)** - you might even be able to make that cup of tea you've been wanting. Just remember to first start running the below cell *before* you put on the kettle! &#9749;

In [ ]:
%%timeit
import condacolab 
!conda install psrchive

**Action 2:** Import more python libraries by runnning the cell below

In [ ]:
import os
import sys
import glob
import random
import subprocess
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import date
sns.set_context('talk')
import psrchive

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

plt.rcParams['figure.figsize'] = (8.0, 8.0)
## The next step randomly chooses a color map from this short list. 
## So the plots could be a different color everytime you rerun the notebook. 
## Feel free to add your own favourite color map here!
my_cmap = random.choice(['viridis', 'magma','gist_heat', 'seismic']) 
print(f'Colormap chosen is: {my_cmap:s}')


**Action 3:** And lastly add some small additional tools from GitHub

In [ ]:
%%capture
!rm -rf ARIWS_tools
!git clone https://github.com/marisageyer/ARIWS_tools/
tools_dir = '/content/ARIWS_tools/scripts/'
sys.path.insert(0, os.path.abspath(tools_dir))
from tools import *

## Step 2: Download the data

The Astronomer on duty for the week, has copied the unanalysed data to an FTP server for you.


**Action 4:** To access the valuable data, you 

1) create a Colab directory "MeerKAT_data" 

2) run *wget* to download the data from the server

3) point the CoLab notebook to the data pathway

4) read in it using psrchive (Finally!)


as in these cells below


In [ ]:
!mkdir -p /content/MeerKAT_data

In [ ]:
!wget ftp://elwood.ru.ac.za/pub/mgeyer/ARIWS/meerkat1.ar -P /content/MeerKAT_data

Now point your CoLab notebook to the data's CoLab pathway.

In [ ]:
mydata_dir = '/content/MeerKAT_data'
datafile = 'meerkat1.ar' ## This is the first mystery data you have obtained!

path_to_data = os.path.join(mydata_dir, datafile)
print(path_to_data)

## Step 2b: and read it in!

&star; Now you can and read it in using psrchive &star;

In [ ]:
data_object = psrchive.Archive_load(path_to_data)
data_object.remove_baseline() 

### &#9758; Note! If you make mistakes down the line with your detective work, 
### then you can come back to **Step 2b** to load the data freshly again. 

## Step 3: Data dimensions

To get a better feel for the data you have, you make notes of the characteristics stored in the **data header**.

A data header contains information about the recorded data, but not the data itself.

**Action 5**: print some of the header information

Notice how the *data_object* which you created above is used to access the header information.

In [ ]:
print(f'The data was captured on {data_object.start_time()} with {data_object.get_telescope()}\n')

obs_duration = data_object.integration_length()
num_chan = data_object.get_nchan()
num_subint = data_object.get_nsubint()
num_bins = data_object.get_nbin()

print(f'And has the following characteristics:\n')
print(f'Observation duration of \t\t\t {obs_duration:.2f} seconds')
print(f'Number of frequency channels\t\t\t {num_chan}')
print(f'Number of subintegrations (time blocks)\t\t {num_subint}')
print(f'Each subintegration (time block) is made up of   {num_bins} data bins')

**Q1:** Using the above information how much **observing time (seconds)** is associated with each subintegration (or time block)?

In [ ]:
time_per_subint =  ## add your computation here. 
print(f'{time_per_subint} seconds')

## Step 4: The observing band
These days MeerKAT boasts two operational receivers on every antenna, each with their own observing frequencies. 

Better check, which observing band was used for the observation you are analysing. 

**Action 6:** You decide the quickest way is to see what frequencies (in MHz) are associated with data you have loaded, and find

In [ ]:
freqs = data_object.get_frequencies()
print(f'{freqs[0]:.2f} to {freqs[-1]:.2f} in MHz')

**Q2**: Which observing band is this? U-band or L-band?

In [ ]:
## Add your answer here

## Step 5: A first plot

Now that you know the dimensions of your data, you decide it's best to have a visual look at it.

If you add all the data in time - the signal will be the brightest. 

*psrchive* has a list of "scrunch" commands to add the data_object in different dimensions:

- fscrunch() -- to integrate across frequency
- tscrunch() -- to integrate across time

Use the `tscrunch()` command to add data in time. 

You can check if it worked correctly, by double checking that the number of subintegrations (time blocks) after you have applied it is **equal to one**. 

**If not**, you've probably used the wrong scrunch command, and it's best to load the data newly by running from Step 2b!


In [ ]:
data_object ## add your scrunching command here to add the data in TIME. The command should end with open brackets, just like the commands in Step 3.

print(f'Your data now has {data_object.get_nsubint():d} subintegration (time blocks) with {data_object.get_nbin():d} data bins')

data = data_object.get_data() ## This command retrieves the data as a numpy array from the data_object (As you've seen the data_object contains more than just the data itself, e.g. the header information in Step 3)
print(np.shape(data))
data = data.reshape(len(freqs),data_object.get_nbin()) ## This gets rid of those extra dimensions in the above shape that only has one column
print(np.shape(data)) ## Now the data is a two-dimensional array with frequency channels, and time bins (a single time block from adding all the data, made up of 1024 data points)

## Let's plot the data
plt.imshow(data, aspect='auto', cmap=my_cmap, origin='lower')
labely = plt.ylabel("Frequency channels")
labelx = plt.xlabel("Time bins")

Double check the number of time blocks,

In [ ]:
print(f'Your data now has {data_object.get_nsubint():d} subintegration (time blocks) with {data_object.get_nbin():d} data bins')

**Q4:** Does your data now have only **one** subintegration (time block)? 

## Step 6: Who goes there?

You're not so sure about this data! That looks a suspiciously **man made**!

Time to zoom in on the contaminated channels and see where exactly they lie. 


**Action 8:** Zoom in on the strange channels in the above plot **by picking y-axis limits.** 

This will help you identify which channels numbers exactly look contaminated. 



In [ ]:
## Pick a window of channel numbers to zoom into
zoom_in_lower =  ## with this as the lower limit 
zoom_in_upper =  ## and this as the upper

plt.imshow(data, aspect='auto', cmap=my_cmap, origin='lower')
plt.ylim(zoom_in_lower, zoom_in_upper)
labely = plt.ylabel("Frequency channels")
labelx = plt.xlabel("Time bins")

**Q5**: Which range of channel numbers do you think are contaminated by artificial signals?

In [ ]:
## add your answer here

## Step 7: removing Radio Frequency Interference (RFI)

Ok, now that you know which are the contaminated channels, it is time to remove them. 

**Action 9:** Choose the range of frequency channels that you want to delete (set to zero) because you think they contain Radio Frequency Interference (RFI) and not astronomical signals


In [ ]:
## Pick the channels to get rid of
RFI_channel_start = ## first channel to remove
RFI_channel_end = ## and remove up to this channel number

## Zero the values in "data" from channel_start to channel_end:
data[RFI_channel_start:RFI_channel_end,:] = 0

Plot the signal after the RFI channels have been removed (set to zero)

In [ ]:
## Plot the signal after having zeroed the RFI frequency channels
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8,10))
ax1.set_title("Datablock after removing frequency channels %d to %d" %(RFI_channel_start, RFI_channel_end))
ax1.imshow(data, aspect='auto', cmap=my_cmap, origin='lower')
labely = ax1.set_ylabel("Frequency channel number")
labelx = ax1.set_xlabel("Time bins")

ax2.set_title("Zoom in: datablock after removing frequency channels %d to %d" %(RFI_channel_start, RFI_channel_end))
ax2.imshow(data, aspect='auto', cmap=my_cmap, origin='lower')
ax2.set_ylim(zoom_in_lower, zoom_in_upper)
labely = ax2.set_ylabel("Frequency channel number")
labelx = ax2.set_xlabel("Time bins")

Now some other signals have popped out! Can you believe that was hiding behind the contaminated data. 

Not bad, detective!

If you see more RFI you can remove even more frequency channels to get a further increase in brightness of the astrophysical signal.

Remember RFI is signatures that effect particular individual channels. 

In [ ]:
## Now zero another set of channels containing RFI
## MG: I have added this so we don't spend too much time on RFI
RFI_channel_start2 = 480 ## add the channel number of the start of the affected region here 
RFI_channel_end2= 500 ## and the channel number of the end of the affected region 
data[RFI_channel_start2:RFI_channel_end2] = 0
## And plot again 

Plot the data again after having removed more RFI frequency channels
You can continue to fine-tune the RFI removal until you are happy with it.
The plotting color scheme should improve as you remove the brightest RFI signals. 

In [ ]:
## Plot the signal after having removed more RFI frequency channels -- if you did
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8,10))
ax1.set_title("Datablock after removing frequency channels %d to %d" %(RFI_channel_start, RFI_channel_end))
ax1.imshow(data, aspect='auto', cmap=my_cmap, origin='lower')
labely = ax1.set_ylabel("Frequency channel number")
labelx = ax1.set_xlabel("Time bins")

ax2.set_title("Zoom in: datablock after removing frequency channels %d to %d" %(RFI_channel_start, RFI_channel_end))
ax2.imshow(data, aspect='auto', cmap=my_cmap, origin='lower')
ax2.set_ylim(zoom_in_lower, zoom_in_upper)
labely = ax2.set_ylabel("Frequency channel number")
labelx = ax2.set_xlabel("Time bins")

## Step 8: Labelling axes to MHz
### Why is the signal so banded? It looks like it is smeared across frequencies.

Since you are happy that you have removed the interference, you decide to make a first proper plot.

With frequency labelling in **MHz** rather than channel numbers.

Luckily you've already created 'freqs' (in Step 4) which contains the frequencies in MHz for each channel. 

**Action 10:** See if you can change the extent of the plot to go from the first frequency channel to the last. 

For now you'll leave the time bins to go from 0 to 1024 (arbitrary units). 

In [ ]:
timestart = 0
timeend = data_object.get_nbin()

lowestfreq = ## fill in the lowest frequency channel in MHz here (using the array 'freqs')
highestfreq = ## fill in the highest frequency channel in MHz here (using the array 'freqs')

plt.figure(figsize=(6,10))
plt.imshow(data, extent=(timestart,timeend,lowestfreq, highestfreq), aspect='auto', cmap=my_cmap, origin='lower')
plt.grid(color='w', ls='dashed') ## adding white grid lines to the plot
plt.ylabel("Frequency (MHz)")
plt.xlabel("Time bins (arbitrary unit)")

Looking at a single smeared signal, you see it appears to 

**arrive at a higher frequency first (time=0), and is then travels to a lower frequency in time ...**

this rings a bell!

**Action 11:** Again, you zoom in to look more closely at a smaller part of the figure. 

Let's look only at a 100 MHz range.
Picking a lower part of the band (lower frequency values) will show the effect better!

In [ ]:
## Pick a 50 MHz frequency range to zoom into, by picking a lower and upper limit
## Picking a lower part of the band (lower frequency values) will show the effect better!
ylim_low_MHz = ## in MHz
ylim_high_MHz =  ## in MHz

plt.figure(figsize=(8,10))
plt.imshow(data, extent=(0,timeend,freqs[0], freqs[-1]), aspect='auto', cmap=my_cmap, origin='lower')
plt.ylabel("Frequency (MHz)")
plt.xlabel("Time (arbitrary unit)")
plt.ylim(ylim_low_MHz,ylim_high_MHz) ## this is where your limit are used

## Step 9: Of course it's a ......



**Q6:** What is causing the smear across frequency?

**Q7:** And why is the signal wrapping?

**Q8:** What astrophysical object has MeerKAT been looking at?

In [ ]:
## add your answers here
## Q6:
## Q7:
## Q8:

#### Your answers from Q7 and Q8 -- have given you a hint at how to transform from your arbitrary units of time to time in seconds!

You suspect you are looking at pulsar data where the signal is wrapping around. That means the time axis represents the number of time samples that MeerKAT has recorded for a given *pulse period*. 

So if you can figure out how fast the data was sampled/recorded - you can compute the pulse period of this data. 

The time resolution of time per bin that the telescope is recording at luckily also lives in the **header** information of the data, and can be obtained by:

In [ ]:
time_resolution = time_per_bin(data_object) ## time per bin in seconds

In [ ]:
print(f'Capturing data every {time_resolution:.8f} seconds or {####} microseconds') ### add a print statement for microseconds 

## Step 10: Calculating the pulse period
You are really zoning in on this mystery now! 

Once you know the pulsar period you are well on your way to figuring out which source this is!

**Action 13:** Compute the period of this periodic source, based on your knowledge of the time associated with each time bin (the time resolution) and the number of time data bins. 


In [ ]:
period =  ## compute the pulse period here
ms_period = period*1000 ## transform the pulse period in seconds to miliseconds

print(f'The pulsar has a rotational period of {period:.5f} seconds')
print(f'or {ms_period:.3f} miliseconds')

## Step 10b: Labelling axes to seconds


Alright now you can also redo the plot of Step 8 and change the x-axis to **pulse period in seconds**, 

or to a single pulse rotation from pulse phase 0 to 1

In [ ]:
time_start = 0 
time_stop = period

fig, ax = plt.subplots(figsize=(8,10))
ax.imshow(data, extent=(time_start,time_stop,lowestfreq, highestfreq), aspect='auto', cmap=my_cmap, origin='lower')
ax.grid(color='w', ls='dashed') ## adding white grid lines to the plot
ax.set_ylabel("Frequency (MHz)")
ax.set_xlabel("Time (seconds)")
ax1 = ax.twiny()
ax1.set_xlabel("Pulse rotation (phase)")

## Step 11: Look up the source 

A list of known pulsars can be found in the *Australia Telescope National Facility's* (ATNF) pulsar catalogue here:

https://www.atnf.csiro.au/research/pulsar/psrcat/

To find a source with a pulse period (P0) close to the computed pulse period, you can use the *condition* and filter for 


`P0 < p_large && P0 > p_small`


such that the pulse period in your data (P0) lies between some values p_small and p_large.

After you have entered your condition, **tick at least "Name" and "P0" before hitting the "Table" button** -- which will display a table of pulsar names and pulse periods based on the condition you entered. 





**Q9:** From the table produced on ATNF which pulsar's data do you think you are looking at?

In [ ]:
## Q9: answer here

## Step 12: Estimate the dispersion measure

To align the pulsar signal, you know you have to get rid of this smearing effect.

For which, paging through your notebook, you have to compute the dispersion measure - which relates the amount of frequency smear for a given time


The delay, $\Delta t$, in seconds, between the arrival times of pulses at frequencies, $\nu_{lo}$ and $\nu_{hi}$ is given by

\begin{equation}
\Delta t = 4.149\times10^3 \biggl(\rm{DM}\biggr) \biggl[\frac{1}{\nu_{lo}^{2}}-\frac{1}{\nu_{hi}^{2}}\biggr] \tag{1}
\end{equation}

where $\nu_{lo}$ and $\nu_{hi}$ are in MHz, 

or 

\begin{equation}
\rm{DM} = \frac{\Delta t}{4.149\times10^3} \frac{\nu_{lo}^{2} \nu_{hi}^{2}}{\nu_{hi}^{2}-\nu_{low}^{2}} \tag{2}
\end{equation}


So if you can estimate the characteristic smearing, or dispersion measure (DM), then you can properly align the pulsar signal.

 

**Action 13:** Looking back at your zoomed-in figure of the frequency smear (Action 11),   

estimate a high frequency where the signal starts (at t=0) and a low frequency value where the signal ends (at t=pulse period), so that you can **use the pulse period as the delay ($\Delta t$) between these frequencies**.

The next cell re-plots your plot in Action 11, so you don't have to scroll up, 

with an added grid lines to improve your ability to estimate values from the frequency axis.

It is important to try to estimate as accurately as possible at which frequencies a single smeared out pulsar signal 

cuts the axis at t=0 and t=period. 

In [ ]:
fig,ax=plt.subplots(figsize=(8,10))
ax.imshow(data, extent=(0,period,freqs[0], freqs[-1]), aspect='auto', cmap=my_cmap, origin='lower')
ax.set_ylabel("Frequency (MHz)")
ax.set_xlabel("Time (seconds)")
ax.yaxis.set_minor_locator(ticker.MultipleLocator(2.0))
ax.grid(which='minor', ls='dashed', alpha=0.8,color='w')
ax.set_ylim(ylim_low_MHz,ylim_high_MHz) ## this is the plotting limits you used before

In [ ]:
## Pick the frequency range of a single smeared siganl 
freq_hi =  ## pick the start of a smeared signal here (in MHz)
freq_low = ## and the end of the smeared signal here (in MHz)

In [ ]:
DM_const = 4149  ## This is known as the dispersion measure constant and has the value 4149 MHz^2 pc^−1 cm^3 s, as you can see in the above calculation
DM = ## compute the DM value based on delta t = pulse period and the frequencies you've picked above, here, using eq. 2
print(f'DM={DM:.2f}')

We can overlay the trend predicted by your DM value on the plot, to see if it looks sensible. 

In [ ]:
## Here we are using the DM value you obtained, and the frequency range which you picked (freq_low to freq_hi)
## to compute the associated predicted time-delay with respect to freq_hi
## Then we will have two arrays to plot (predicted_t, freq_array)
freq_array = np.linspace(freq_low,freq_hi,100)
predicted_t = DM*DM_const*(1/freq_array**2 - 1/freq_hi**2) #this uses eq1

In [ ]:
fig,ax=plt.subplots(figsize=(8,12))
ax.imshow(data, extent=(0,period,freqs[0], freqs[-1]), aspect='auto', cmap=my_cmap, origin='lower')
ax.set_ylabel("Frequency (MHz)")
ax.set_xlabel("Time (seconds)")
ax.yaxis.set_minor_locator(ticker.MultipleLocator(2.0))
ax.grid(which='minor', ls='dashed', alpha=0.8,color='w')
ax.set_ylim(ylim_low_MHz,ylim_high_MHz) ## this is the plotting limits you used before
ax.plot(predicted_t,freq_array,'c') ## depending on the color scheme you might want to change the color of this line

If it doesn not look good, see if you can improve on the frequency ranges you picked. 

## Step 13: Correct the dispersion
Now let's correct the dispersion for each channel by using the computed DM value.

You bring up an old schematic you have to remind you how the dedispersion process works. 

In [ ]:
img='/content/ARIWS_tools/scripts/dedispersion.png'
from IPython.core.display import Image, display
display(Image(img, width=600, unconfined=True))

Luckily, you have a function that you've written before which computes the required shift for each frequency channel in seconds. 

**Action 14:** Since your data is an indexed numpy array here, you just have to write that shift in seconds into a shift of data bins.

In [ ]:
def rotate_array(arr, nbins):
    return [arr[(nbins + i) % len(arr)] for i in range(len(arr))]


def correct_dispersion(data, freqs,DM, period, printout=True):
    print(f'Correcting dispersion using a DM value of {DM:.4f} pc cm$^{-3}$')
    print(f'---------------------------------------------------------------')
    nchan, nbin = np.shape(data)
    DMcorr_data=np.zeros((nchan,nbin),dtype=float)

    center_freq=np.median(freqs)

    for chan in range(nchan):
 
        data_chan = data[chan]
        freq_chan = freqs[chan]
        if printout==True:
          print(f'To correct the dispersion of frequency channel {chan:d} ({freq_chan:.2f} MHz), ',end='')
        shift_sec = DM_const*DM*(1/freq_chan**2 - 1/center_freq**2)

        shift_bins =  ## add your computation for the shift in bins here. In other words transform the shift in seconds (shift_sec) to a shift in bins. 
                      ## remember bins should be an integer, so after transforming from seconds to bins use int() 
        shift_bins_mod = np.mod(shift_bins,1024) 
        if printout==True:
          print(f'we shift the data by {1000*shift_sec:.2f} ms or {shift_bins_mod:d} bins')

        # rotate the data by the number of bins
        data_shifted=rotate_array(data_chan,shift_bins_mod)

        DMcorr_data[chan,:]=data_shifted
    
    return DMcorr_data

**Action 15:** Run the function to correct dispersion using your computed DM value. 

In [ ]:
DMcorr_data=correct_dispersion(data,freqs,DM,period)

**Action 16:** Let's see how well that worked by plotting the corrected signal.

In [ ]:
plt.figure(figsize=(8,10))
plt.imshow(DMcorr_data, extent=(0,period,freqs[0], freqs[-1]), aspect='auto', cmap=my_cmap, origin='lower')
plt.ylabel("Frequency (MHz)")
plt.xlabel("Time (sec)")
plt.ylim(ylim_low_MHz,ylim_high_MHz) ## you can play around with these plotting limits to zoom in and out
plt.grid()

## Step 13b: Fine tune the Dispersion Measure (DM)

If you estimated the dispersion measure correctly the signal should now align across frequencies, producing **vertical** pulses.

**Action 17:** We can find an accurate DM value by looping over a range of DMs and seeing what looks best

*Hint:* Only make small changes (< 2.0 pc cm$^{-3}$) at a time. The DM value is very sensitive!

If a single smeared signal is still arriving **earlier at higher frequencies** than lower ones -- you need to slightly **increase** the DM value you computed. 

If a single smeared signal is arriving **earlier at lower frequencies** than higher ones -- you have overestimated the DM, and need to slightly **decrease** the DM value you computed.

### If you are struggling to obtain a good DM value, you can also loop over values to test a good DM range.

### This is a critical step in how searches for pulsars are conducted!

In [ ]:
## Set up a DM range. The code block will pick 16 values from this start to end range and generate plots. 
DM_start = ## select a DM start value
DM_stop= ## select a DM stop value

sns.set_context('paper')  
fig,axs=plt.subplots(4,4, figsize=(10,10))
fig.subplots_adjust(hspace = .5, wspace=.5)
axs = axs.ravel()
idx = 0

for dm in np.linspace(DM_start,DM_stop,16):

  DM = dm ## use a value slighlty higher or lower than your first estimated DM (from Action 13) to help fine-tune its value
  DMcorr_data=correct_dispersion(data,freqs,DM,period,printout=False)  ## Setting printout to "False" surpresses all the print statements from before for a cleaner screen.

  #plot the corrected data
  axs[idx].imshow(DMcorr_data, extent=(0,period,freqs[0], freqs[-1]), aspect='auto', cmap=my_cmap, origin='lower')
  axs[idx].set_title('DM: %.3f' %dm)
  idx+=1

#### Finalised DM value
**Make sure you use your best and final DM value to produce the final version of DMcorr_data** 

In [ ]:
## Make sure you use your best and final DM value to produce the final version of 
BestDM = ####
DMcorr_data=correct_dispersion(data,freqs,BestDM,period,printout=False)

## Step 14: Finally, the added pulse profile!

**Action 18**: Adding all the frequency channels after the dispersion is corrected, gives you the integrated stable pulsar profile

In [ ]:
average_profile = np.sum(###, axis=0) ## sum the DM corrected array here across frequency (hence axis=0)

plt.figure(figsize=(10,5))
plt.title("Dispersion corrected profile")
plt.plot(np.linspace(0,1,1024), average_profile)
plt.show()

plt.figure(figsize=(10,5))
print(f'\nGood thing we did the dedispersion, without it the signal would look much worse:\n')
plt.title("No dispersion correction")
plt.plot(np.linspace(0,1,1024), np.sum(data, axis=0))
xlab=plt.xlabel("Pulse rotation (phase)")

### Can you see how bad the signal looks like without dispersion correction. 

### What an interesting pulse shape the resulting pulsar has - we see two peaks! 

**Q10:**Why do you think we are seeing two peaks?



## Step 15: Summarise 

Time for this detective to summarise what's been learned from the data so far. 


In [ ]:
## Firstly this is data of a pulsar, 
## At the MeerKAT band
val_band =
## and you think the source name is
val_pulsarname =
## It has a pulse period of 
val_pulseperiod = ms_period ## in ms
## The source has a DM value of
val_DM =
## After an observation of length
val_obslength = data_object.integration_length()


print(f'Summary for MeerKAT data {val_band} data analysed on {date.today()}: \n')
print(f'Pulsar: \t PSR {val_pulsarname}\n')
print(f'Pulse period: \t {val_pulseperiod:.5f} ms \n')
print(f'DM value: \t {val_DM:.4f} pc cm-3\n')
print(f'Obs duration: \t {val_obslength:.2f} seconds\n')

### Ok, so the averaged pulse profile is what you obtained after having added the observing time together, as well as all the frequency channels, and correcting for the dispersion.

### &#128161; but since pulsars are interesting clocks ... you think it might be worth checking how the pulsar behaves with time

## Step 16: Let's see this clock tick -- **load in time resolved data**
To observe what is happening with the pulse signal you will need to load the data newly, since previously you added the timeblocks together using tscrunch()

So back to loading in the original data ...


In [ ]:
data_object2 = psrchive.Archive_load(path_to_data)
data_object2.remove_baseline()

## You rerun some of the commands you used in Step 4, to remind yourself of the data dimensions
print(f'And has the following characteristics:')
print(f'Number of frequency channels\t\t\t {data_object2.get_nchan()}')
print(f'Number of subintegrations (time blocks)\t\t {data_object2.get_nsubint()}')
print(f'Each subintegration (time block) is made up of \t {data_object2.get_nbin()} data bins')

time_per_subint = data_object2.integration_length()/data_object2.get_nsubint()
print(f'Time of each subintegration: \t\t\t {time_per_subint:.2f} seconds')

This time you will add all the frequency channels together from the start, and just study how the pulsar changes with time

You do so with the **fscrunch()** command

In [ ]:
data_object2.dedisperse() ## This corrects for dedispersion based on header information (so an effective alternative to computing it yourself!!)
print("Note - this dedisperse function is what pulsar astronomers actually use to dedisperse data from a known pulsar with a known DM value")
data_object2.fscrunch()
data_time = data_object2.get_data().reshape(data_object2.get_nsubint(),data_object2.get_nbin())
print(np.shape(data_time)) ## the only dimensions of the data now are the number of time blocks (subintegrations) and phase bins

## Step 17: Watch the timeseries as a movie to see how the pulsar behaves with time


This next cell just sets up the plotting canvas for the animation

In [ ]:
%%capture
# Set up the figure canvas for the movie
fig,ax = plt.subplots(1, figsize=(12,5))
title = ax.text(0.45,0.85, "", bbox={'facecolor':'w', 'alpha':0.5, 'pad':5},
                transform=ax.transAxes, ha="center")
ph, = ax.plot(data_time[130])
txt = ax.text(100, 0.004, "Max peak value",color='r')
#ax.axis('off')
ax.set_ylim(-0.001, 0.01)

**Action 20:** Run the animation. Note that it will take some time to load **(~ 3min)**

In [ ]:
# function for drawing each frame
def singleframe(i):
  ph.set_ydata(data_time[i])
  ph.set_xdata(np.linspace(0,1024,1024))

  title.set_text(u" Time block = {}, Max. value = {:.4f}".format(i, np.max(data_time[i])))
  xx = np.linspace(0,1024,len(data_time))
  max_points = [np.max(data_time[i]),np.max(data_time[i-1])] 
  maxtrend = ax.plot([xx[i], xx[i-1]], max_points, 'r-o', markersize=4,alpha=0.6)                                    
  return ph, title, maxtrend
  
# create the animation
ani=animation.FuncAnimation(fig, singleframe, range(5, len(data_time)))
ani

**Q11:** Do you see any changes in the pulse profile with time?

**Q12:** If so, when do the changes appear? 

**Q13:** And how long do they last?

**Action 22:** Let's also make a 2D plot of how the pulse shape changes.
We'll use the observing time to change the axis from time blocks (subintegrations) to observing time (seconds)

In [ ]:
##The full observing time of the data is given by, 
obstime = data_object2.integration_length()
print(f'The full observing time is {obstime:.2f} seconds')
obs_start = 0
obs_end = obstime

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(data_time,extent=(0,1,obs_start, obs_end), aspect='auto', cmap='magma', origin='lower')
plt.ylabel("Time (sec)")
plt.xlabel("Pulse phase")
plt.grid()

**Q14:** What do you see when plotting the pulse intensity as a function of time in this 2D plot? Does it agree with what you observed in the animation?

**Action 23:** Estimate the duration for which the pulsar appears to dim (in seconds) using either the animation or the 2D plot output

In [ ]:
dimming_duration = ## in seconds

## &star; &star; &star; You've got to know more about this odd dimming of the pulsar! &star; &star;

#### You know that MeerKAT can record at even higher time rates than the data you have. 

#### Time to ring the operator on duty. 


### &#9742; Ring ring ...

#### If she can observe pulsar J0737-0737 for you at a higher recording rate, than maybe you can learn more about what is going on in the gap ...  almost like Jocelyn Bell-Burnell running her penchart recorder at higher speeds :)

#### The operator on duty picks up: you provide her with the pulsar name and your dispersion measure to set the coherent dedispersion back-end. 




## Step 18: The **high time resolution** data is here! 

Now that you already know most of the process it will be easily to check out. 
Load in the data and rerun some of the commands you used in Step 4 to investigate the data dimensions

Time to download it using *wget* as before

In [ ]:
!wget ftp://elwood.ru.ac.za/pub/mgeyer/ARIWS/meerkat2_highres.ar -P /content/MeerKAT_data

In [ ]:
datafile3 = 'meerkat2_highres.ar'

path_to_data = os.path.join(mydata_dir, datafile3)
print(path_to_data)
data_object3 = psrchive.Archive_load(path_to_data)

data_object3.remove_baseline()

## You rerun some of the commands you used in Step 4, to remind yourself of the data dimensions
print(f'And has the following characteristics:')
print(f'Number of frequency channels\t\t\t {data_object3.get_nchan()}')
print(f'Number of subintegrations (time blocks)\t\t {data_object3.get_nsubint()}')
print(f'Duration of the observation\t\t\t {data_object3.integration_length():.2f} seconds')

time_per_subint = data_object3.integration_length()/data_object3.get_nsubint()
print(f'Time of each subintegration \t\t\t {time_per_subint:.2f} seconds')

**Q14:** How does this time per block or subintegration compare to the lower time resolution data you had previously?

In [ ]:
## answer here

In [ ]:
data3 = data_object3.get_data().reshape(data_object3.get_nsubint(),data_object3.get_nbin())

In [ ]:
## The full observing time of these data are given by, 
obstime_highresdata = data_object3.integration_length()
print(f'The full observing time is {obstime_highresdata:.2f} seconds. So a much shorter observation than the previous one. Just zooming in on that part where the pulsar light dims.')

## Step 19: Plot the **high time resolution** data

Investigate the high time resolution data, again using a 2D plot of pulse period or pulse phase on the x-axis and observing time on the y-axis.

In [ ]:
obs_start= 0 ## the observation starting time in seconds
obs_end= obstime_highresdata ## and the observation ending time in seconds (similar to the second plot of Step 18) 

plt.figure(figsize=(8,8))
plt.imshow(data3,aspect='auto', extent=(0,1, obs_start,  obs_end), cmap=my_cmap, origin='lower')
plt.ylabel("Observing time (sec)")
plt.xlabel("Pulse phase")

## &#129427;  ...     Zebra-stripes in the gap where the pulsar light dims!
## What could that mean?
### By plotting the data added across the pulse, you can see the ossicilations more clearly. 


In [ ]:
fig, ax = plt.subplots(2,1,figsize=(18,8))
fig.subplots_adjust(hspace=0.5)

ax[0].set_title("The cumulative change in pulse intensity as a function of observing time")
ax[0].plot(np.linspace(0,obstime_highresdata,data_object3.get_nsubint()),np.sum(data3[:,320:420], axis=1))
ax[0].set_xlabel("Observing time (seconds)")

ax[1].set_title("Zoom in to see the spacings between the fluctuations")
ax[1].plot(np.linspace(0,obstime_highresdata,data_object3.get_nsubint()),np.sum(data3[:,320:420], axis=1))
ax[1].set_xlim(40,70)
ax[1].set_xlabel("Observing time (seconds)")


# Here we add more detailed gridlines to help you see if it is straight!
ax[1].xaxis.set_minor_locator(ticker.MultipleLocator(0.5))
ax[1].xaxis.set_major_locator(ticker.MultipleLocator(2))

ax[1].grid(which='major', axis='both', alpha=0.6,color='k', ls='solid')
ax[1].grid(which='minor', axis='x', alpha=0.4,color='k', ls='dashed')

## Step 20: The period of these new peaks

### The zoomed in plot you've made above shows how during that dimming episode of the pulsar, the intensity of the observed pulsar light is fluctuation.

**Action 24:** Compute the time between peaks in the osscillations during the dimming of the pulsar

In [ ]:
peak1 = ## choose a time at which one peak occurs
peak2 = ## and when the next peak occurs
time_between_peaks = ## use peak1 and peak2 to compute the time between peaks in seconds

### &#128269; $\quad$ The status of your detective notes, 

In [ ]:
print(f'During a MeerKAT observation of {val_band} data of PSR {val_pulsarname},\n')
print(f'which has a pulse period of {val_pulseperiod:.2f} ms, \n')
print(f'the pulsar light appeared to dim for about {dimming_duration} seconds. \n')
print(f'You requested higher time resolution data to see what exactly is going on during that dimming episode ... \n')
print(f'And found another periodic signal within the gap with a period of around {time_between_peaks} seconds! \n')

print('What a crazy system!\n')
print('What could be going on?')

## Step 23: Time to play hangman to help you understand!

In [ ]:
hangman()

After playing hangman you can also view this video on the SARAO YouTube channel (https://youtu.be/QVxaXg8EQYo) to learn more about the the system.